In [23]:
import numpy as np
import pandas as pd

In [24]:
class Hmm:
    def __init__(self,file,k=3):
        self.words = None #set of unique words
        self.tag_word_count = None # dict((tag,word),count)
        self.transmissions = None # dict((tag_u,tag_v),count)
        self.count = None # dict(tag,count)
        self.read_file(file,k)
        self.tags = set(self.count.keys())
        
        self.word_ls = tuple(self.words)#tuple cause immutable
        self._tag_ls = tuple(self.tags)
        #move #START# to first row and #END# to last row
        ls = list(self._tag_ls)
        ls.remove('#START#')
        ls.remove('#END#')
        ls.insert(0,'#START#')
        ls.append('#END#')
        self.tag_ls = tuple(ls)
        
        self.make_matrix()
    
    def make_matrix(self):
        tag_length = len(self.tag_ls)
        transition_matrix = np.zeros((tag_length,tag_length))
        
        for i in range(tag_length):
            for j in range(tag_length):
                tag_u = self.tag_ls[i]
                tag_v = self.tag_ls[j]
                transition_matrix[i][j] = self.transmissions[(tag_u,tag_v)]/self.count[tag_u]
        self.transition_matrix = pd.DataFrame(transition_matrix,index=self.tag_ls,columns=self.tag_ls)
        
        word_length = len(self.word_ls)
        em_matrix = np.zeros((tag_length,word_length))
        for i in range(tag_length):
            for j in range(word_length):
                tag = self.tag_ls[i]
                word = self.word_ls[j]
                em_matrix[i][j] = self.tag_word_count[(tag,word)]/self.count[tag]
        self.em_matrix = pd.DataFrame(em_matrix,index=self.tag_ls,columns=self.word_ls)
        pass
    
    def read_file(self,file,k):
        from collections import defaultdict
        seq = ['#START#']
        f = open(file,'r',encoding='UTF-8')
        tag_word_ls = []
        word_count = defaultdict(int)
        for line in f:
            split = line.split(' ')
            if len(split)<2:
                #this is a line break
                seq.append('#END#')
                seq.append('#START#')
                continue
            word,tag = split
            word = word.strip()
            tag = tag.strip()
            tag_word_ls.append([tag,word])
            word_count[word]+=1
            seq.append(tag)
        f.close()
        
        #Emissions
        for i in range(len(tag_word_ls)):
            tag,word = tag_word_ls[i]
            if word_count[word]<k:
                tag_word_ls[i] = [tag,'#UNK#']
        tag_word_count = defaultdict(int)
        
        words = []
        for tag,word in tag_word_ls:
            tag_word_count[tag,word]+=1
            words.append(word)
        self.words = set(words)
        self.tag_word_count= tag_word_count
        
        #Transistions
        del seq[-1] #delete last item from the list
         #print(seq)
        trans_dict = defaultdict(int)
        count_u = defaultdict(int)
        for i in range(len(seq)-1):
            tag_u = seq[i]
            count_u[tag_u] += 1 # need to count #END# too
            if tag_u == "#END#":
                continue
            #if u is not #END# we count the transmission 
            tag_v = seq[i+1]
            if (tag_u =="#START#" and tag_v == "#END#"):
                #check for empty blank lines at the end and dont count them
                print('these are blank lines')
                count_u["#START#"] -= 1 #remove additional start
                break
            trans_dict[(tag_u,tag_v)] += 1
        self.transmissions = trans_dict
        self.count = count_u

In [25]:
EN = Hmm('./EN/train')

In [26]:
EN.transition_matrix.loc['#START#','B-VP']

0.018661098786376094

In [27]:
EN.transition_matrix.values[0,:]

array([0.00000000e+00, 0.00000000e+00, 1.30497194e-03, 0.00000000e+00,
       0.00000000e+00, 5.42868328e-02, 0.00000000e+00, 1.04397755e-03,
       0.00000000e+00, 0.00000000e+00, 3.26242986e-03, 0.00000000e+00,
       2.25760146e-02, 0.00000000e+00, 1.08704163e-01, 1.41850450e-01,
       0.00000000e+00, 2.60994389e-04, 0.00000000e+00, 1.86610988e-02,
       6.48049067e-01, 0.00000000e+00, 0.00000000e+00])

In [28]:
EN.transition_matrix

,#START#,I-UCP,B-INTJ,I-SBAR,B-PRT,B-ADVP,I-CONJP,B-LST,I-PP,I-VP,...,I-NP,B-PP,O,B-UCP,B-CONJP,I-INTJ,B-VP,B-NP,I-ADVP,#END#
#START#,0.0,0.00,0.001305,0.000000,0.000000,0.054287,0.000000,0.001044,0.000000,0.000000,...,0.000000,0.108704,0.141850,0.000000,0.000261,0.000000,0.018661,0.648049,0.000000,0.000000
I-UCP,0.0,0.75,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000
B-INTJ,0.0,0.00,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.038462,0.615385,0.000000,0.000000,0.192308,0.076923,0.038462,0.000000,0.000000
I-SBAR,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.020833,0.958333,0.000000,0.000000
B-PRT,0.0,0.00,0.000000,0.000000,0.000000,0.029915,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.241453,0.158120,0.000000,0.000000,0.000000,0.042735,0.506410,0.000000,0.000000
B-ADVP,0.0,0.00,0.000000,0.000000,0.000281,0.016269,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.170547,0.265358,0.000000,0.000561,0.000000,0.215989,0.210379,0.086957,0.000842
I-CONJP,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.281250,0.000000,0.000000,0.000000,...,0.000000,0.109375,0.015625,0.000000,0.000000,0.000000,0.156250,0.437500,0.000000,0.000000
B-LST,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I-PP,0.0,0.00,0.000000,0.000000,0.000000,0.004484,0.000000,0.000000,0.071749,0.000000,...,0.000000,0.062780,0.040359,0.000000,0.000000,0.000000,0.031390,0.784753,0.000000,0.000000
I-VP,0.0,0.00,0.000000,0.000000,0.023329,0.037602,0.000000,0.000000,0.000000,0.327887,...,0.000000,0.148341,0.056698,0.000000,0.000197,0.000000,0.008269,0.355350,0.000000,0.000098


In [29]:
EN.transition_matrix.loc['#START#','B-NP']

0.6480490669450607

In [331]:
def log(m):
    m = np.clip(m, 1e-32, None)
    x = np.log(m)
    return x

In [380]:
def vertebi_k(word_arr,Hmm,k=7):
    """
    Followed pseudocode here
    https://en.wikipedia.org/wiki/Viterbi_algorithm#Pseudocode
    """
    S = Hmm.tag_ls[1:-1] #set of all possible tags remove #START# and #STOP#
    
    A = Hmm.transition_matrix.values[1:-1,1:-1] # A(tag_u_vector,tag_v)
    B = Hmm.em_matrix[1:-1] # B(tag_u->word)
    
    T = len(S) # Total number unique tags
    N = len(word_arr) # Length of sentence make sure no #START# and #STOP#
    
    T1 = np.zeros((T,N,k)) #probability table of most possible path to node i.e. store scores of each node
    T2 = np.zeros((T,N,k)) # Table of paths where the ith row stores highest scoring paths to T1[i,j]
    
    #Handle first word and base case at the same time
    word = word_arr[0]
    if word not in Hmm.words:
        word = '#UNK#'
        
    T1[:,0,:] = log(Hmm.transition_matrix.loc['#START#'][1:-1].values*B[word].values)[:,None]
    
    #Note A is vector operation
    # Fill up each column by using previous column
    # j is position of word
    for j in range(1,N):
        # i is position of tag
        #ignore #START# and #END# tag when looping
        word = word_arr[j]
        if word not in Hmm.words:
            word = '#UNK#'
        #note A(S,tag_u gives a vector)
        
        edge_matrix = (np.repeat(T1[:,j-1,None],T,axis=1)+np.repeat(log(A*B[word].values)[:,:,None],k,axis=2))
        
        #map all edges from 1 layer to the next
        for edge in range(T):
            #get top k values
            #top argsort and get top k
            current_col = edge_matrix[:,edge,:]
            top_k_scores = np.sort(current_col.ravel())[::-1][:k]
            top_k_backpointers = np.dstack(np.unravel_index(current_col.ravel().argsort(),current_col.shape))[0][::-1][:k]
            print(top_k_backpointers)
            T1[edge][j] = top_k_scores
            T2[edge][j] = top_k_backpointers
#             print(top_k_scores)
#             print(top_k_backpointers)

        
    #handle last word to #END#
    #no emission of #END# 
#     print(T1)
#     print(T2)

    best_row = np.argmax(T1[:,N-1]+log(Hmm.transition_matrix['#END#'].values[1:-1]))
    ans=[]
    curr_index = best_row
    ans.append(S[curr_index])
    for j in range(N-1,0,-1):
        prev_index = T2[int(curr_index)][j]
        ans.append(S[int(prev_index)])
        curr_index = prev_index
#         print(S[j])
    ans = ans[::-1]
#     print(T1)
    return ans

In [381]:
word = "Trump is the best president in the world".split(' ')
len(word)

8

In [382]:
vertebi_k(word,EN)

[[20  6]
 [ 5  1]
 [ 7  3]
 [ 7  2]
 [ 7  1]
 [ 7  0]
 [ 6  6]]


ValueError: could not broadcast input array from shape (7,2) into shape (7)

In [12]:
LANG = ['AL','CN','EN','SG']
eval_params = lambda lang: {'devin':f'./{lang}/dev.in','devout':f'./{lang}/dev.p3.out','ground_truth':f'./{lang}/dev.out','trainfile':f'./{lang}/train'}

In [13]:
import os
def pred_out(devin,devout,ground_truth,trainfile):
    H = Hmm(trainfile)
    file_object = open(devin, "r",encoding='UTF-8',)
    ls=[[]]
    index=0
    test=[]
    for line in file_object:
        test.append(line.strip())
        if (line.strip()==""):
            ls.append([])
            index+=1
        else:
            ls[index].append(line.strip())
    ls.pop(-1)
    df = pd.DataFrame(test, columns = ['Word'])
    
    from tqdm.notebook import tqdm
    predict=[]
    for i in tqdm(ls):
        for j in vertebi(i,H):
            predict.append(j)
        predict.append("")
    df['Tag'] = predict
    
    df.to_csv(devout, sep=" ", index=False, header=False)
    
    if os.name == 'nt':#if it is on windows
        !python ./EvalScript/evalResult.py {ground_truth} {devout}
    else:
        !python3 ./EvalScript/evalResult.py {ground_truth} {devout}

In [14]:
for lang in LANG:
    print(lang)
    pred_out(**eval_params(lang))
    print('---------------------------------')

AL




#Entity in gold data: 8408
#Entity in prediction: 8498

#Correct Entity : 6740
Entity  precision: 0.7931
Entity  recall: 0.8016
Entity  F: 0.7974

#Correct Sentiment : 6087
Sentiment  precision: 0.7163
Sentiment  recall: 0.7240
Sentiment  F: 0.7201
---------------------------------
CN




#Entity in gold data: 1478
#Entity in prediction: 712

#Correct Entity : 307
Entity  precision: 0.4312
Entity  recall: 0.2077
Entity  F: 0.2804

#Correct Sentiment : 210
Sentiment  precision: 0.2949
Sentiment  recall: 0.1421
Sentiment  F: 0.1918
---------------------------------
EN




#Entity in gold data: 13179
#Entity in prediction: 13060

#Correct Entity : 11079
Entity  precision: 0.8483
Entity  recall: 0.8407
Entity  F: 0.8445

#Correct Sentiment : 10651
Sentiment  precision: 0.8155
Sentiment  recall: 0.8082
Sentiment  F: 0.8118
---------------------------------
SG




#Entity in gold data: 4537
#Entity in prediction: 3008

#Correct Entity : 1665
Entity  precision: 0.5535
Entity  recall: 0.3670
Entity  F: 0.4414

#Correct Sentiment : 1036
Sentiment  precision: 0.3444
Sentiment  recall: 0.2283
Sentiment  F: 0.2746
---------------------------------


In [62]:
a = np.random.rand(3,3)

In [63]:
a

array([[0.58554262, 0.15672213, 0.05409078],
       [0.25433011, 0.13347871, 0.17217374],
       [0.37576197, 0.75958957, 0.09796061]])

In [79]:
np.repeat(a[:,:,None],7,axis=2)

array([[[0.58554262, 0.58554262, 0.58554262, 0.58554262, 0.58554262,
         0.58554262, 0.58554262],
        [0.15672213, 0.15672213, 0.15672213, 0.15672213, 0.15672213,
         0.15672213, 0.15672213],
        [0.05409078, 0.05409078, 0.05409078, 0.05409078, 0.05409078,
         0.05409078, 0.05409078]],

       [[0.25433011, 0.25433011, 0.25433011, 0.25433011, 0.25433011,
         0.25433011, 0.25433011],
        [0.13347871, 0.13347871, 0.13347871, 0.13347871, 0.13347871,
         0.13347871, 0.13347871],
        [0.17217374, 0.17217374, 0.17217374, 0.17217374, 0.17217374,
         0.17217374, 0.17217374]],

       [[0.37576197, 0.37576197, 0.37576197, 0.37576197, 0.37576197,
         0.37576197, 0.37576197],
        [0.75958957, 0.75958957, 0.75958957, 0.75958957, 0.75958957,
         0.75958957, 0.75958957],
        [0.09796061, 0.09796061, 0.09796061, 0.09796061, 0.09796061,
         0.09796061, 0.09796061]]])

In [204]:
b= np.arange(28,1,-1)

In [205]:
b = b.reshape(3,3,3)

In [206]:
for i in b:
    print('\n')
    for j in i:
        print('',end= '\t')
        for k in j:
            print(k,end= ' ')



	28 27 26 	25 24 23 	22 21 20 

	19 18 17 	16 15 14 	13 12 11 

	10 9 8 	7 6 5 	4 3 2 

In [207]:
b = np.max(b,axis=2)

In [221]:
b

array([[28, 25, 22],
       [19, 16, 13],
       [10,  7,  4]])

In [224]:
b.reshape(9)

array([28, 25, 22, 19, 16, 13, 10,  7,  4])

In [284]:
b[1][1] =100

In [270]:
np.dstack(np.unravel_index(b.ravel().argsort(),b.shape))

array([[[2, 2],
        [2, 1],
        [2, 0],
        [1, 2],
        [1, 0],
        [0, 2],
        [0, 1],
        [0, 0],
        [1, 1]]], dtype=int64)

In [285]:
b

array([[  4,   7,  10],
       [ 13, 100,  22],
       [ 25,  28, 100]])

In [297]:
np.sort(b.ravel())

array([  4,   7,  10,  13,  22,  25,  28, 100, 100])

In [279]:
np.sort(b.ravel())

array([  4,   7,  10,  13,  19,  22,  25,  28, 100])

In [289]:
b.ravel().argsort()

array([0, 1, 2, 3, 5, 6, 7, 4, 8], dtype=int64)

In [295]:
np.sort(b.ravel())

array([  4,   7,  10,  13,  22,  25,  28, 100, 100])